In [1]:
import pyenzyme as pe

In [2]:
# Data was simulated using the model s0 * a + b
# Multiple measurements with different offsets
# Replicate data is for s1

In [3]:
from src import Model, LoadEnzymeML, ParameterEstimation 

In [6]:
# Build graph
enzmldoc = pe.EnzymeMLDocument.fromFile("Ethanol_Formation_Simulated.omex")
load_node = LoadEnzymeML(path="Ethanol_Formation_Simulated.omex")
load_node.write_graph(run=True)

for index, reaction_obj in enumerate(enzmldoc.reaction_dict.values()):
    model_node = Model(
        data=load_node if index == 0 else model_node,
        equation=reaction_obj.model.equation,
        reaction=reaction_obj.id,
        name=reaction_obj.name,
        config={
            parameter.name: {
                "value": parameter.value,
                "unit": parameter.unit,
                "initial_value": parameter.value,
                "upper": parameter.value + 10,
                "lower": 0.0000001
            }
            for parameter in reaction_obj.model.parameters
        }
    )
    model_node.write_graph(run=True)
    
estimation_node = ParameterEstimation(data=model_node, thinlayer="pysces")
estimation_node.write_graph(run=True)

2022-05-16 08:24:27,091 (WARNING): Running DVC command: 'dvc run -n LoadEnzymeML ...'
2022-05-16 08:24:28,723 (WARNING): Running DVC command: 'dvc run -n Pyruvate Decarboxylation ...'
2022-05-16 08:24:30,120 (WARNING): Running DVC command: 'dvc run -n Ethanol formation ...'
2022-05-16 08:24:33,486 (WARNING): Running DVC command: 'dvc run -n ParameterEstimation ...'


In [7]:
!dvc dag

      +--------------+       
      | LoadEnzymeML |       
      +--------------+       
              *              
              *              
              *              
+--------------------------+ 
| Pyruvate Decarboxylation | 
+--------------------------+ 
              *              
              *              
              *              
    +-------------------+    
    | Ethanol formation |    
    +-------------------+    
              *              
              *              
              *              
  +---------------------+    
  | ParameterEstimation |    
  +---------------------+    
